In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import xgboost as xg
import numpy as np
from sklearn.metrics import log_loss, accuracy_score

test = pd.read_csv('/kaggle/input/otto-group-product-classification-challenge/test.csv', index_col = 0)
train = pd.read_csv('/kaggle/input/otto-group-product-classification-challenge/train.csv', index_col = 0)
sample = pd.read_csv('/kaggle/input/otto-group-product-classification-challenge/sampleSubmission.csv')


(144368, 93)


Created a predictor matrix

In [3]:
dataset_train = train.values
x = dataset_train[:,0:93] 
y = dataset_train[:,93] 


Label encoder to assign labels to categorical values

In [4]:
label_encoder = LabelEncoder()
label_encoder.fit(y)
le_y = label_encoder.transform(y)


Train test split

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, le_y, test_size = 0.33, random_state = 7)

Creating and training a model

In [6]:
xgb = xg.XGBClassifier(learning_rate =0.23, n_estimators=520, max_depth=7, min_child_weight=3, subsample=0.7, 
                       colsample_bytree=0.7, nthread=4, seed=42, objective='multi:softprob')

xgb.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.7,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.23, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=150,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)

Prediction probability

In [7]:
pred = xgb.predict_proba(x_test)
print("Validation data logloss: {}".format(log_loss(y_test, pred)))

Validation data logloss: 0.49067266574967194


Accuracy test

In [8]:
pred = xgb.predict(x_test)
accu = accuracy_score(y_test, pred)
print("Accuracy: %.2f%%" % (accu * 100.0))

Accuracy: 81.25%


Submit predictions in the sample submission format for Kaggle

In [12]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
dataset_test = test.values



prediction_sub = xgb.predict(dataset_test)
X_sub = np.array(prediction_sub).reshape(-1,1)
onehot_encoder = OneHotEncoder(sparse = False)
submission_file = onehot_encoder.fit_transform(X_sub)

submission_file_df = pd.DataFrame(submission_file, 
                                  columns = ['Class_1','Class_2','Class_3','Class_4','Class_5','Class_6',
                                            'Class_7','Class_8','Class_9'], index = test.index)


submission_file_df.to_csv("submission.csv")

In [13]:
submission_file_df

,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
id,,,,,,,,,
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
144364,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144365,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
144366,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
